In [60]:
# 必要なパッケージを呼び出す
using LinearAlgebra
using NLsolve
using Statistics
using Printf
using BenchmarkTools

In [61]:
# MATLABではグローバル変数を使って関数への変数受け渡しを行う
# Julia/Pythonではコンストラクタを用いる

# ギリシャ文字の使用は不可とする
struct Model{TI<:Integer, TF<:AbstractFloat, TV<:Vector}

    # カリブレーション
    beta::TF          # 割引因子
    gamma::TF         # 相対的危険回避度(異時点間の代替の弾力性の逆数)
    alpha::TF         # 資本分配率
    delta::TF         # 固定資本減耗(delta=1.0のときは解析解が存在)  
    #定常状態の値
    ykss::TF
    kss::TF
    yss::TF
    css::TF    
    #グリッドに関するパラメータ
    nk::TI            # グリッドの数
    kmax::TF          # 資本グリッドの最大値
    kmin::TF          # 資本グリッドの最小値
    kgrid::TV         # 資本グリッド
    #補間に関するパラメータ
    T::Array{TF,2}    # 基底行列
    invT::Array{TF,2} # 基底行列の逆関数
    #時間反復法に関するパラメータ
    maxiter::TI       # 繰り返し計算の最大値
    tol::TF           # 許容誤差

end

In [62]:
function polybas(kmin,kmax,Np,kgrid)
    """
    ----------------------------------------
    === 基底関数の行列を再帰的に求める関数 ===
    ----------------------------------------
    <input>
    ・kmin: 状態変数のグリッドの最小値
    ・kmax: 状態変数のグリッドの最大値
    ・Np: 多項式の次数-1   
    ・kgrid: 状態変数のグリッド(Ng個)
    <output>
    ・T: 基底関数の行列(NgxNp)
    (Ng個の評価点でNp-1次のチェビシェフ多項式で近似する)
    """
    Ng = length(kgrid) #グリッドの数
    x = (2/(kmax-kmin))*(kgrid .- kmin) .- 1 # グリッドを[-1,1]の範囲に変換

    # 基底関数の行列(NgxNp)を再帰的に求める
    T = zeros(Ng,Np)
    T0 = ones(Ng)
    T1 = x
    T2 = 2 .* x .* T1 .- T0
    T[:,1] = T1
    T[:,2] = T2
    
    for i in 3:Np-1
        T[:,i] = 2 .* x .* T[:,i-1] .- T[:,i-2]
    end

    T = [T0 T[:,1:(Np-1)]]

    return T

end

polybas (generic function with 1 method)

In [63]:
function polygrid(kmin,kmax,N)
    """
    -----------------------------------------------
    === チェビシェフ多項式における評価点を返す関数 ===
    -----------------------------------------------
    <input>
    ・kmin: 状態変数のグリッドの最小値
    ・kmax: 状態変数のグリッドの最大値
    ・N: 状態変数のグリッド数
    <output>
    ・k: 状態変数のグリッド
    """
    # チェビシェフ極値点
    temp = collect(range(0,N-1,length=N))
    x = -cos.((pi/(N-1)).*temp)
    
    # チェビシェフゼロ点
    #x = -cos.((pi/2/(N-1) .* (2*temp.-1)))
    #x[1] = 0.0

    #x([-1,1])からk([kmin,kmax])に変換
    k = 0.5.*(kmax-kmin).*(x.+1) .+ kmin

    return k

end

polygrid (generic function with 1 method)

In [64]:
function EulerEq_cheb(x,m,capital,theta)
    """
    ----------------------------------------------
    === オイラー方程式に代入した際の残差を返す関数 ===
    ----------------------------------------------
    <input>
    ・x: 今期の消費水準
    ・m: パラメータ等を格納したコンストラクタ
    (※Matlabコードではglobal変数を用いているが、Python/Juliaコード上ではコンストラクタにパラメータを格納しているので、
    コンストラクタを明示的に関数の引数にして、パラメータを関数内で呼び出している。)
    ・capital: 今期の資本保有量
    ・theta: チェビシェフ補間した際のパラメータの値
    <output>
    ・res: オイラー方程式に代入した際の残差
    """
    cons = x[1] # 最適化問題を解くために引数をベクトルにする必要がある。
    wealth = (capital^m.alpha) + (1-m.delta)*capital
    kprime = wealth - cons
    kprime = max(m.kgrid[1],kprime) # トリック: k'は正の値しか取らない

    # 次期の政策関数を多項式補間する
    # Tはk=kprimeで評価した基底関数
    T = polybas(m.kmin,m.kmax,m.nk,[kprime]) # 次期の資本グリッドにおける基底関数の行列を求める
    # 多項式の係数thetaを基底関数に掛けて近似値を求める
    cnext = (T*theta)[1]

    # オイラー方程式の残差を求める（u'(c)をmu_CRRA関数を用いて計算している）
    res = mu_CRRA(cons,m.gamma) .- m.beta*mu_CRRA(cnext,m.gamma)*(m.alpha*kprime.^(m.alpha-1).+(1-m.delta))

    return res

end

EulerEq_cheb (generic function with 1 method)

In [65]:
function CRRA(cons,gamma)
    """
    -----------------------
    === CRRA型効用関数 ===
    -----------------------
    <inputs>
    ・cons: 消費量
    ・gamma: 相対的リスク回避度(異時点間の代替弾力性の逆数)
    <output>
    ・utility: consとgamma の下での効用水準
    """

    if gamma != 1
        utility = (cons^(1-gamma)) / (1-gamma)
    else
        utility = log(cos)
    end

    return utility

end

CRRA (generic function with 1 method)

In [66]:
function mu_CRRA(cons,gamma)
    """
    --------------------------
    === CRRA型限界効用関数 ===
    --------------------------
    <inputs>
    ・cons: 消費量
    ・gamma: 相対的リスク回避度(異時点間の代替弾力性の逆数)
    <output>
    ・consとgamma の下での限界効用水準
    """

    mu = cons^(-gamma)

    return mu
    
end

mu_CRRA (generic function with 1 method)

In [67]:
function nti_cheb(m)
    """
    ---------------------
    === TI を解く関数 ===
    ---------------------
    <input>
    ・m: パラメータ等を含む構造体
    <output>
    ・cfcn0: 政策関数
    ・dif: 繰り返し誤差
    """
    
    # 収束の基準
    it = 1;         # ループ・カウンター
    dif2 = 1.0;     # 政策関数の繰り返し誤差
    tolfun = 1e-10; # nlsolveのオプション(最適化の許容誤差)  

    # STEP 1(b): 政策関数の初期値を当て推量
    # 解析解 (for k'=g(k))
    p_true = m.beta*m.alpha*(m.kgrid.^m.alpha);

    # 政策関数の初期化
    cfcn0 = copy(m.kgrid);
    cfcn1 = zeros(m.nk);

    # 繰り返し誤差を保存する変数を設定 
    dif = zeros(2,m.maxiter);

    # STEP 4: 政策関数を繰り返し計算
    while (it<m.maxiter) && (dif2>m.tol)

        # 補間はあらかじめグリッド上で計算した基底関数invTにより行う
        # thetaは多項式の係数
        theta = m.invT * cfcn0

        for i in 1:m.nk

            capital = m.kgrid[i]
            wealth = (capital^m.alpha) + (1.0-m.delta)*capital

            # Juliaの最適化関数(nlsolve)を使って各グリッド上の政策関数の値を探す
            # オイラー方程式の誤差をゼロにするようなcの値を求める
            Euler = x-> EulerEq_cheb(x,m,capital,theta);
            # 最適化の初期値は古い政策関数の値
            res = nlsolve(Euler,[cfcn0[i]],ftol=tolfun);
            cons = res.zero[1];
            cfcn1[i] = cons;
            kprime = wealth - cons;
    
        end

        # 繰り返し計算誤差を確認
        dif2 = maximum(abs.(cfcn1-cfcn0));

        # 収束途中の繰り返し計算誤差を保存
        dif[2,it] = dif2;

        # 政策関数をアップデート
        cfcn0 = copy(cfcn1);

        @printf "iteration index: %1d \n" it  
        @printf "policy function iteration error: %1.6f \n" dif2 
        flush(stdout)

        it += 1

    end

    return cfcn0, dif
end

nti_cheb (generic function with 1 method)

In [68]:
function calcerr(m,cfcn0)
    """
    --------------------------------------
    === オイラー方程式誤差を測定する関数 ===
    --------------------------------------
    <input>
    ・m: パラメータ等を格納したコンストラクタ
    (※Matlabコードではglobal変数を用いているが、Python/Juliaコード上ではコンストラクタにパラメータを格納しているので、
    コンストラクタを明示的に関数の引数にして、パラメータを関数内で呼び出している。)
    ・cfcn0: (収束した)政策関数
    <output>
    ・err: オイラー方程式誤差
    """

    theta = m.invT*cfcn0
    # 元のグリッドではオイラー方程式の誤差はゼロになるため、グリッドを細かくとる
    kgrid_err = collect(range(m.kmin,m.kmax,length=(m.nk-1)*10+1))
    T = polybas(m.kmin,m.kmax,m.nk,kgrid_err)
    cons = T*theta
    LHS = mu_CRRA.(cons,m.gamma)

    kp = (kgrid_err.^m.alpha) .+ (1.0-m.delta).*kgrid_err .- cons
    T = polybas(m.kmin,m.kmax,m.nk,kp)
    cnext = T*theta
    rent = m.alpha.*(kp.^(m.alpha-1.0)) .- m.delta
    RHS = m.beta .* (1.0.+rent) .* mu_CRRA.(cnext,m.gamma)

    err = (RHS./LHS) .- 1.0

    return err

end

calcerr (generic function with 1 method)

In [69]:
# メインファイル

# カリブレーション
beta = 0.96;  # 割引因子
gamma = 1.0;  # 相対的リスク回避度(異時点間の代替の弾力性の逆数)
alpha = 0.40; # 資本分配率
delta = 1.0;  # 固定資本減耗(delta=1.0のときは解析解が存在)

# 定常状態の値
ykss = (1.0/beta-1.0+delta)/alpha;
kss = ykss^(1.0/(alpha-1.0));
yss = ykss*kss;
css = yss-delta*kss;

# グリッドに関するパラメータ
nk = 9;         # グリッドの数
kmax = 1.2*kss; # 資本グリッドの最大値
kmin = 0.8*kss; # 資本グリッドの最小値
kgrid = polygrid(kmin,kmax,nk); # チェビシェフ評価点

# 補間に関するパラメータ
T = polybas(kmin,kmax,nk,kgrid); # 基底行列
invT = inv(T);                   # 基底行列の逆関数

# 時間反復法に関するパラメータ
maxiter = 1000; # 繰り返し計算の最大値
tol = 1e-8;     # 許容誤差

# 収束の基準に関するパラメータ
it = 1;         # ループ・カウンター
dif2 = 1.0;     # 政策関数の繰り返し誤差
tolfun = 1e-10; # newtonのオプション(最適化の許容誤差)

In [70]:
norms = zeros(3,2);
times = zeros(3,2);

nkvec = [3,5,9];

for i in 1:3

    # STEP 1(a): グリッド生成
    nk = nkvec[i];
    kgrid = polygrid(kmin,kmax,nk);
    T = polybas(kmin,kmax,nk,kgrid);
    invT = inv(T);
    
    m = Model(beta,gamma,alpha,delta,ykss,kss,yss,css,nk,kmax,kmin,kgrid,T,invT,maxiter,tol);

    # time iteration
    cfcn0, dif = nti_cheb(m);
    times[i,1] = @elapsed nti_cheb(m); # 非効率・2重計算/ tik toc みたいなのないのか

    err = calcerr(m,cfcn0);
    norms[i,:] = log10.([mean(abs.(err)), maximum(abs.(err))]')

end

times[:,2] = times[:,1]./times[1,1]


display("Euler equation errors")
display(round.(norms,digits=2))

display("Elapsed time")
display(round.(times,digits=2))

"Euler equation errors"

3×2 Matrix{Float64}:
 -3.5   -3.23
 -5.8   -5.49
 -7.68  -7.68

"Elapsed time"

3×2 Matrix{Float64}:
 0.0  1.0
 0.0  2.07
 0.0  5.07